In [11]:
library(naturalsort)
library(data.table)
weights <- read.csv('./data/snap_velcro_weights.csv')
weights <- weights[7,2:5]
flst <- naturalsort(list.files('./data/gene_counts_exon/')) # File names are same for each directory (lung, colon, brain)
flst <- flst[flst!='out.log']

In [12]:
chroms <- c(1:22,'X','Y')
for (i in 1:24) {
    load(paste0('./data/gene_counts_exon/',flst[i]))
    rownames(counts) <- gnames
    #--- Below the weights of each Snap Velcro category are multiplied by the counts
    counts[,1] <- counts[,1]*weights$closed_tight
    counts[,2] <- counts[,2]*weights$closed_loose
    counts[,3] <- counts[,3]*weights$open_tight
    counts[,4] <- counts[,4]*weights$open_loose
    counts <- rowSums(counts)
    if (i==1) {
        out <- data.table(chrom=rep(chroms[i],length(counts)),gene_id=as.vector(gnames),counts=counts,length=as.vector(lens),type=rep(0,length(counts)))
    } else {
        tmp <- data.table(chrom=rep(chroms[i],length(counts)),gene_id=as.vector(gnames),counts=counts,length=as.vector(lens),type=rep(0,length(counts)))
        out <- rbind(out,tmp)
    }
}
tail(out)
gene_pd <- out

chrom,gene_id,counts,length,type
Y,ENSG00000172352,55.77855,2675,0
Y,ENSG00000183795,18.60601,1436,0
Y,ENSG00000187191,82.03821,3516,0
Y,ENSG00000205916,91.65669,4348,0
Y,ENSG00000185894,18.60601,1436,0
Y,ENSG00000172288,55.20674,2625,0


In [13]:
# Add gene symbol field
load('./data/genes.rda')
genes <- genes[order(genes$gene_id),]
gene_pd <- gene_pd[order(gene_pd$gene_id),]
identical(genes$gene_id,gene_pd$gene_id)
gene_pd$gene_sym <- genes$gene_sym

[1] TRUE

In [14]:
# Weight probabilities of ENd and ENi insertion
gene_pd$p <- 0.9*(gene_pd$counts/sum(gene_pd$counts))+0.1*(gene_pd$length/sum(gene_pd$length))

# For male genome, double the probability of chroms. 1-22 genes being hit, then renormalize
gene_pd_m <- gene_pd
gene_pd_m$p[!(gene_pd_m$chrom %in% c('X','Y'))]=gene_pd_m$p[!(gene_pd_m$chrom %in% c('X','Y'))]*2
gene_pd_m$p <- gene_pd_m$p/sum(gene_pd_m$p)

# For female genome, double the probability of chroms. 1-23 genes being hit, and set chroms. Y genes prob to 0, then renormalize
gene_pd_f <- gene_pd
gene_pd_f$p[gene_pd_f$chrom!='Y']=gene_pd_f$p[gene_pd_f$chrom!='Y']*2
gene_pd_f$p[gene_pd_f$chrom=='Y']=0
gene_pd_f$p <- gene_pd_f$p/sum(gene_pd_f$p)


In [15]:
save(gene_pd_m,gene_pd_f,file='./data/gene_pd_exon.rda')